In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os
print(f'''Env Variables
LANGCHAIN_TRACING_V2: {os.environ["LANGCHAIN_TRACING_V2"]}
LANGCHAIN_PROJECT: {os.environ["LANGCHAIN_PROJECT"]}
LANGCHAIN_ENDPOINT: {os.environ["LANGCHAIN_ENDPOINT"]}
''')

Env Variables
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: machine-learning-workshop
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com



## LangSmithのトレースの基本
traceableデコレータを使用することで、任意の関数の引数と返り値をLangSmithで確認できるようになる。

In [20]:
from IPython.display import display, Markdown
from langsmith import traceable
import openai

openai_client = openai.Client()

@traceable
def format_prompt(question):
    return [
        {
            "role": "system",
            "content": "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です.",
        },
        {
            "role": "user",
            "content": f"{question}"
        }
    ]

@traceable(run_type="llm")
def invoke_llm(messages):
    return openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )

@traceable
def parse_output(response):
    return response.choices[0].message.content

@traceable
def run_pipeline():
    messages = format_prompt("ウェブサイトの回遊率")
    response = invoke_llm(messages)
    return parse_output(response)

display(Markdown(run_pipeline()))

```sql
SELECT
  user_id,
  COUNT(DISTINCT page_id) AS pages_visited,
  COUNT(DISTINCT session_id) AS sessions,
  COUNT(DISTINCT page_id) / COUNT(DISTINCT session_id) AS page_views_per_session
FROM
  `your_dataset.your_table`
GROUP BY
  user_id
```

openaiとのやり取りの可観測にするラッパー `wrap_openai` を使うと詳細な情報を簡単に取得可能になる。

In [32]:
from langsmith.wrappers import wrap_openai

wrap_openai_client = wrap_openai(openai.Client())

@traceable(name="run_pipeline with wrap_openai")
def run_pipeline_with_wrap_llm():
    messages = format_prompt("ウェブサイトの回遊率")
    response = wrap_openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )
    return parse_output(response)

display(Markdown(run_pipeline_with_wrap_llm()))

```sql
SELECT
  user_id,
  COUNT(DISTINCT session_id) AS total_sessions,
  COUNT(DISTINCT CASE WHEN page_viewed THEN session_id END) AS sessions_with_page_views,
  (COUNT(DISTINCT CASE WHEN page_viewed THEN session_id END) / COUNT(DISTINCT session_id)) AS bounce_rate
FROM
  `your_dataset.your_table`
GROUP BY
  user_id
```

LCEL (LangChain Expression Language) を使えば、LangSmithでの観測が楽にできる。

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です."),
    ("human", "{question}")
])
output_parser = StrOutputParser()

chain = prompt | model | output_parser
display(Markdown(chain.invoke("ウェブサイトの回遊率")))

```
# まずは全体のセッション数と、1ページ以上訪れたセッション数を取得
WITH total_sessions AS (
  SELECT COUNT(*) as total_sessions
  FROM `project.dataset.sessions`
),
pages_per_session AS (
  SELECT sessionId, COUNT(*) as num_pages
  FROM `project.dataset.page_views`
  GROUP BY sessionId
),
sessions_with_multiple_pages AS (
  SELECT COUNT(*) as sessions_with_multiple_pages
  FROM pages_per_session
  WHERE num_pages > 1
)
# 回遊率を計算（1ページ以上訪れたセッション数 / 全体のセッション数）
SELECT sessions_with_multiple_pages / total_sessions as bounce_rate
FROM total_sessions, sessions_with_multiple_pages
```

このクエリは、ウェブサイトの回遊率を計算するためのものです。回遊率とは、ユーザーがウェブサイトの複数のページを訪れたセッションの割合を指します。

まず、全体のセッション数を`total_sessions`サブクエリで取得します。次に、各セッションで訪れたページ数を`pages_per_session`サブクエリで取得します。そして、1ページ以上訪れたセッション数を`sessions_with_multiple_pages`サブクエリで取得します。

最後に、1ページ以上訪れたセッション数を全体のセッション数で割ることで、回遊率を計算します。

なお、このクエリはBigQueryのStandard SQL構文を使用しています。また、このクエリはサンプルであり、実際のデータスキーマによっては修正が必要です。

In [88]:
import textwrap

from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate

ls_client = Client() # LangSmithのクライアント

# 作成するデータセット
dataset_name = "SQL Samples"

# データセットがあれば削除
if ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.delete_dataset(dataset_name=dataset_name)

dataset = ls_client.create_dataset(dataset_name, description="ML Workshop用のサンプルクエリ")

# データセットにexampleを保存
ls_client.create_examples(
    inputs=[
        {"question": "MAUを取得"},
        {"question": "新規ユーザ数の推移"},
    ],
    outputs=[
        {"query": textwrap.dedent("""
           SELECT
               COUNT(DISTINCT user_id) AS monthly_active_users
           FROM
               `your_dataset.user_activities`
           WHERE
               activity_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AND CURRENT_DATE()
        """),
         "tables": ["user_activities"]
        },
        {"query": textwrap.dedent("""
            SELECT
                signup_date,
                COUNT(user_id) AS new_users
            FROM
                `your_dataset.user_activities`
            GROUP BY
                signup_date
            ORDER BY
            　　 signup_date
        """),
         "tables": ["user_activities"]
        },
    ],
    dataset_id=dataset.id,
)

In [89]:
dataset

Dataset(name='SQL Samples', description='ML Workshop用のサンプルクエリ', data_type=<DataType.kv: 'kv'>, id=UUID('76f610c2-59c6-4981-a77c-06e715b017e1'), created_at=datetime.datetime(2024, 6, 22, 8, 48, 7, 11550, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 6, 22, 8, 48, 7, 11550, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None)

example_countが0となっているので、LangSmith Clientを使ってdatasetを読み直す

In [90]:
ls_client.read_dataset(dataset_name=dataset_name)

Dataset(name='SQL Samples', description='ML Workshop用のサンプルクエリ', data_type=<DataType.kv: 'kv'>, id=UUID('76f610c2-59c6-4981-a77c-06e715b017e1'), created_at=datetime.datetime(2024, 6, 22, 8, 48, 7, 11550, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 6, 22, 8, 48, 7, 11550, tzinfo=datetime.timezone.utc), example_count=2, session_count=0, last_session_start_time=None)

In [91]:
dataset.url

'https://smith.langchain.com/o/bd14a154-65e7-52b4-bdce-b9a16d5e3513/datasets/76f610c2-59c6-4981-a77c-06e715b017e1'

In [92]:
ls_client.create_examples(
    inputs=[
        {"question": "月ごとのCV数の推移"},
    ],
    outputs=[
        {"query": textwrap.dedent("""
           SELECT
               FORMAT_TIMESTAMP('%Y-%m', conv_date) AS conversion_month,
               COUNT(conv_id) AS conversions
           FROM
               `your_dataset.your_table`
           WHERE
               conv_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) AND CURRENT_DATE()
           GROUP BY
               conversion_month
           ORDER BY
               conversion_month
        """),
         "tables": ["user_activities"]
        },
    ],
    dataset_id=dataset.id,
)

exampleを増やしたことにより、datasetのバージョンも変更されている

In [93]:
ls_client.read_dataset(dataset_name=dataset_name).example_count

3

In [94]:
# datasetに保存されているexampleの一覧
for example in ls_client.list_examples(dataset_name=dataset_name):
    print(f'''
question: {example.inputs["question"]}
query: {example.outputs["query"]}
    ''')


question: 月ごとのCV数の推移
query: 
SELECT
    FORMAT_TIMESTAMP('%Y-%m', conv_date) AS conversion_month,
    COUNT(conv_id) AS conversions
FROM
    `your_dataset.your_table`
WHERE
    conv_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) AND CURRENT_DATE()
GROUP BY
    conversion_month
ORDER BY
    conversion_month

    

question: MAUを取得
query: 
SELECT
    COUNT(DISTINCT user_id) AS monthly_active_users
FROM
    `your_dataset.user_activities`
WHERE
    activity_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AND CURRENT_DATE()

    

question: 新規ユーザ数の推移
query: 
SELECT
    signup_date,
    COUNT(user_id) AS new_users
FROM
    `your_dataset.user_activities`
GROUP BY
    signup_date
ORDER BY
　　 signup_date

    


## LangSmith Evaluation
### Custom Evaluation

In [104]:
# inputsにexampleが1つずつ渡される
def predict(inputs: dict) -> dict:
    model = ChatOpenAI(model="gpt-4")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です."),
        ("human", "{question}, tableはuser_activitiesを使います.")
    ])
    output_parser = StrOutputParser()
    llm = prompt | model | output_parser
    return {"output": llm.invoke(inputs)}

# Custom Evaluation
def must_have_user_activities(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    print(f"run id: {run.id}\n")
    required = example.outputs.get("tables") or [] # outputsのキー (tables) と合わせる
    print(required)
    print(prediction)
    score = all(phrase in prediction for phrase in required) # scoreは自分で定義したものでよい
    return {"key":"must_have_user_activities", "score": score, "comment": "comment test"} # key, score, commentを返す

experiment_results = evaluate(
    predict,
    data=dataset_name, # The data to predict and grade over
    evaluators=[must_have_user_activities], # The evaluators to score the results
    experiment_prefix="ml-workshop", # A prefix for your experiment names to easily identify them
    metadata={
      "version": "1.0.0",
    },
)

View the evaluation results for experiment: 'ml-workshop-4bccaae5' at:
https://smith.langchain.com/o/bd14a154-65e7-52b4-bdce-b9a16d5e3513/datasets/76f610c2-59c6-4981-a77c-06e715b017e1/compare?selectedSessions=f515d07c-4a3a-4624-a4f4-5507ca6cb52a




0it [00:00, ?it/s]

run id: d9be2638-5c51-4cb0-ba2a-bac0eb8a9605
run id: 96ca519b-1bd8-4ed3-9ac9-7ff00a3547f5

['user_activities']
```
SELECT
  DATE(created_at) AS date,
  COUNT(DISTINCT user_id) AS new_users
FROM
  `project.dataset.user_activities`
WHERE
  created_at = first_seen_at
GROUP BY
  date
ORDER BY
  date
```
run id: 83dcf60e-3ead-4dc0-9e9e-9b3956976c91


['user_activities']
```
SELECT 
  EXTRACT(YEAR FROM timestamp) AS year,
  EXTRACT(MONTH FROM timestamp) AS month,
  COUNT(DISTINCT user_id) AS MAU
FROM 
  `project.dataset.user_activities`
GROUP BY 
  year, 
  month
ORDER BY 
  year DESC, 
  month DESC
```
['user_activities']
SELECT 
  FORMAT_TIMESTAMP('%Y-%m', TIMESTAMP_SECONDS(time)) AS month,
  COUNT(*) AS cv_count
FROM 
  `user_activities`
WHERE 
  activity = 'CV'
GROUP BY 
  month
ORDER BY 
  month ASC
